In [2]:
#Import Packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold,train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import StratifiedKFold,KFold,GridSearchCV
from sklearn.feature_selection import RFECV,RFE,f_regression,mutual_info_regression,VarianceThreshold,SelectFromModel,mutual_info_regression
from sklearn.feature_extraction.image import grid_to_graph
from sklearn import feature_selection
from sklearn.cluster import FeatureAgglomeration
from sklearn.linear_model import BayesianRidge


from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from math import sqrt
from sklearn.metrics import r2_score

import eli5
from eli5.sklearn import PermutationImportance

import shutil
import tempfile
from scipy import linalg, ndimage
from joblib import Memory as mem

C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [17]:

df = pd.read_csv('2ex1.csv',header=0)
df1 = pd.read_csv('1ex1.csv',header=0)

#All features
X = df[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','DS_DSDT','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','DS_Fuel_Meter','DS_Slew_Force','DS_Bkt_Force','DS_Dipper_Force','DS_BoomR_Force','DS_BoomL_Force','DS_CylinderBoomLiftL','DS_CylinderBoomLiftR','DS_CylinderDipperLift','DS_CylinderBucketLift','DS_Slew','DS_Bucket_Angle','DS_Dipper_Angle','DS_Boom_Angle','DS_Slew_Angle','DS_BoomCylinderPressureA','DS_BoomCylinderPressureB','DS_DipperCylinderPressureA','DS_DipperCylinderPressureB','DS_BucketCylinderPressureA','DS_BucketCylinderPressureB','DS_Boom_zdd','DS_Boom_ydd','DS_Boom_xdd','DS_Dipper_zdd','DS_Dipper_ydd','DS_Dipper_xdd','DS_Bkt_zdd','DS_Bkt_ydd','DS_Bkt_xdd','DS_Bktwx','DS_Bktwy','DS_Bktwz','DS_Dipperwx','DS_Dipperwy','DS_Dipperwz','DS_Boomwx','DS_Boomwy','DS_Boomwz','DS_Upperwx','DS_Upperwy','DS_Upperwz']]
y = df[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]

#Removed correlated
#X = df[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD4','DS_DSDT','SoilTransferSensor2 MassTot','Collisionsensor_Ground isColliding','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','DS_Slew_Force','DS_Bkt_Force','DS_Dipper_Force','DS_BoomR_Force','DS_BoomL_Force','DS_CylinderBoomLiftL','DS_CylinderBoomLiftR','DS_CylinderDipperLift','DS_CylinderBucketLift','DS_Slew','DS_Bucket_Angle','DS_Dipper_Angle','DS_Boom_Angle','DS_Slew_Angle','DS_BoomCylinderPressureA','DS_BoomCylinderPressureB','DS_DipperCylinderPressureA','DS_DipperCylinderPressureB','DS_BucketCylinderPressureA','DS_BucketCylinderPressureB','DS_Boom_ydd','DS_Boom_xdd','DS_Dipper_ydd','DS_Dipper_xdd','DS_Bkt_zdd','DS_Bkt_ydd','DS_Bkt_xdd','DS_Bktwx','DS_Bktwy','DS_Bktwz','DS_Dipperwx','DS_Dipperwy','DS_Dipperwz','DS_Boomwx','DS_Boomwy','DS_Boomwz','DS_Upperwx','DS_Upperwy','DS_Upperwz']]
#y = df[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X)
X = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y)
y = pd.DataFrame(scaler_y)

X_train,X_test,y_train,y_test = train_test_split(X,y)


C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [24]:
#Define model for Variance-Based Importance

model = Sequential()
model.add(Dense(50, activation = 'relu',input_shape=(57,)))
model.add(Dense(100,activation='relu'))
model.add(Dense(4,activation='sigmoid'))#Changed to sigmoid to allow 0 to 1

model.compile(loss='mse',optimizer='adam')

model.fit(X_train,y_train,batch_size=50,epochs=200,verbose=2,validation_data=(X_test,y_test),shuffle=False) #Changed shuffle to true


Train on 2139 samples, validate on 713 samples
Epoch 1/200
 - 8s - loss: 0.0455 - val_loss: 0.0238
Epoch 2/200
 - 0s - loss: 0.0187 - val_loss: 0.0155
Epoch 3/200
 - 0s - loss: 0.0127 - val_loss: 0.0113
Epoch 4/200
 - 0s - loss: 0.0098 - val_loss: 0.0093
Epoch 5/200
 - 0s - loss: 0.0084 - val_loss: 0.0082
Epoch 6/200
 - 0s - loss: 0.0075 - val_loss: 0.0074
Epoch 7/200
 - 0s - loss: 0.0068 - val_loss: 0.0067
Epoch 8/200
 - 0s - loss: 0.0063 - val_loss: 0.0061
Epoch 9/200
 - 0s - loss: 0.0058 - val_loss: 0.0057
Epoch 10/200
 - 0s - loss: 0.0054 - val_loss: 0.0053
Epoch 11/200
 - 0s - loss: 0.0050 - val_loss: 0.0049
Epoch 12/200
 - 0s - loss: 0.0047 - val_loss: 0.0046
Epoch 13/200
 - 0s - loss: 0.0044 - val_loss: 0.0044
Epoch 14/200
 - 0s - loss: 0.0042 - val_loss: 0.0042
Epoch 15/200
 - 0s - loss: 0.0041 - val_loss: 0.0040
Epoch 16/200
 - 0s - loss: 0.0039 - val_loss: 0.0038
Epoch 17/200
 - 1s - loss: 0.0037 - val_loss: 0.0037
Epoch 18/200
 - 0s - loss: 0.0036 - val_loss: 0.0035
Epoch 19

Epoch 153/200
 - 0s - loss: 8.8681e-04 - val_loss: 0.0011
Epoch 154/200
 - 0s - loss: 8.8037e-04 - val_loss: 0.0011
Epoch 155/200
 - 0s - loss: 8.7609e-04 - val_loss: 0.0011
Epoch 156/200
 - 0s - loss: 8.6941e-04 - val_loss: 0.0011
Epoch 157/200
 - 0s - loss: 8.6675e-04 - val_loss: 0.0011
Epoch 158/200
 - 0s - loss: 8.6312e-04 - val_loss: 0.0011
Epoch 159/200
 - 0s - loss: 8.6235e-04 - val_loss: 0.0011
Epoch 160/200
 - 0s - loss: 8.5431e-04 - val_loss: 0.0011
Epoch 161/200
 - 0s - loss: 8.4578e-04 - val_loss: 0.0011
Epoch 162/200
 - 0s - loss: 8.4479e-04 - val_loss: 0.0011
Epoch 163/200
 - 0s - loss: 8.3696e-04 - val_loss: 0.0011
Epoch 164/200
 - 0s - loss: 8.3081e-04 - val_loss: 0.0011
Epoch 165/200
 - 0s - loss: 8.2582e-04 - val_loss: 0.0011
Epoch 166/200
 - 0s - loss: 8.2333e-04 - val_loss: 0.0011
Epoch 167/200
 - 0s - loss: 8.1995e-04 - val_loss: 0.0011
Epoch 168/200
 - 0s - loss: 8.1272e-04 - val_loss: 0.0011
Epoch 169/200
 - 0s - loss: 8.1042e-04 - val_loss: 0.0010
Epoch 170/200


In [25]:
# Leave-One-Feature-Out LOFO
def LeaveOneFeatureOut(model, X, Y):
    OneOutScore = []
    n = X.shape[0]
    for i in range(0,X.shape[1]):
        newX = X.copy()
        newX[:,i] = 0 #np.random.normal(0,1,n)
        OneOutScore.append(model.evaluate(newX, Y, batch_size=2048, verbose=0))
    OneOutScore = pd.DataFrame(OneOutScore[:])
    ordered = np.argsort(-OneOutScore.iloc[:,0])
    return(OneOutScore, ordered)

In [26]:
L1FO = LeaveOneFeatureOut(model,X_train.values,y_train.values)

In [27]:
pd.set_option('display.max_rows', 500) 
print(L1FO)

(           0
0   0.002911
1   0.001752
2   0.001810
3   0.004777
4   0.001815
5   0.002069
6   0.002653
7   0.002360
8   0.004571
9   0.005720
10  0.003713
11  0.001952
12  0.003700
13  0.003073
14  0.004897
15  0.007257
16  0.007131
17  0.002769
18  0.008681
19  0.001626
20  0.001815
21  0.002221
22  0.003824
23  0.001789
24  0.006718
25  0.001425
26  0.005883
27  0.001330
28  0.003651
29  0.006090
30  0.002607
31  0.002885
32  0.003001
33  0.004730
34  0.005916
35  0.009873
36  0.001595
37  0.001947
38  0.001576
39  0.002309
40  0.001455
41  0.007016
42  0.004050
43  0.001860
44  0.001914
45  0.025937
46  0.005091
47  0.006926
48  0.007986
49  0.003074
50  0.011184
51  0.001486
52  0.004112
53  0.055747
54  0.003833
55  0.005093
56  0.006752, 0     53
1     45
2     50
3     35
4     18
5     48
6     15
7     16
8     41
9     47
10    56
11    24
12    29
13    34
14    26
15     9
16    55
17    46
18    14
19     3
20    33
21     8
22    52
23    42
24    54
25    22
26    10
2

In [70]:
#from google.colab import files
#uploaded = files.upload()
df = pd.read_csv('2ex1.csv',header=0)
df.head()
df1 = pd.read_csv('1ex1.csv',header=0)
df2= pd.read_csv('2ex2.csv',header=0)
df3= pd.read_csv('2ex3.csv',header=0)
df4= pd.read_csv('2ex4.csv',header=0)
df5= pd.read_csv('2ex5.csv',header=0)
df6= pd.read_csv('2ex6.csv',header=0)
df7= pd.read_csv('2ex7.csv',header=0)
df8= pd.read_csv('2ex8.csv',header=0)
df9= pd.read_csv('2ex9.csv',header=0)
df10= pd.read_csv('2ex10.csv',header=0)
df11= pd.read_csv('2ex11.csv',header=0)
df12= pd.read_csv('2ex12.csv',header=0)
df13= pd.read_csv('2ex13.csv',header=0)
df14= pd.read_csv('2ex14.csv',header=0)
df15= pd.read_csv('2ex15.csv',header=0)

X_10 = df[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y_10 = df[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X10=MinMaxScaler()
scaler_X10 = scaler_X10.fit_transform(X_10)
X_10 = pd.DataFrame(scaler_X10)
scaler_y10=MinMaxScaler()
scaler_y10 = scaler_y10.fit_transform(y_10)
y_10 = pd.DataFrame(scaler_y10)

X_pred10 = df1[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y_actual10 = df1[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X_pred10)
X_pred10 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y_actual10)
y_actual10 = pd.DataFrame(scaler_y)

X2 = df2[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y2 = df2[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X2)
X2 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y2)
y2 = pd.DataFrame(scaler_y)

X3 = df3[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y3 = df3[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X3)
X3 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y3)
y3 = pd.DataFrame(scaler_y)

X4 = df4[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y4 = df4[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X4)
X4 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y4)
y4 = pd.DataFrame(scaler_y)

X5 = df5[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y5 = df5[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X5)
X5 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y5)
y5 = pd.DataFrame(scaler_y)

X6 = df6[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y6 = df6[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X6)
X6 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y6)
y6 = pd.DataFrame(scaler_y)

X7 = df7[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y7 = df7[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X7)
X7 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y7)
y7 = pd.DataFrame(scaler_y)

X8 = df8[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y8 = df8[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X8)
X8 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y8)
y8 = pd.DataFrame(scaler_y)

X9 = df9[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y9 = df9[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X9)
X9 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y9)
y9 = pd.DataFrame(scaler_y)

X10 = df10[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y10 = df10[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X10)
X10 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y10)
y10 = pd.DataFrame(scaler_y)

X11 = df11[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y11 = df11[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X11)
X11 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y11)
y11 = pd.DataFrame(scaler_y)

X12 = df12[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y12 = df12[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X12)
X12 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y12)
y12 = pd.DataFrame(scaler_y)

X13 = df13[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y13 = df13[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X13)
X13 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y13)
y13 = pd.DataFrame(scaler_y)

X14 = df14[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y14 = df14[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X14)
X14 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y14)
y14 = pd.DataFrame(scaler_y)

X15 = df15[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz']]
y15 = df15[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X15)
X15 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y15)
y15 = pd.DataFrame(scaler_y)

#X_pred = df1.drop(df1.ix[:,69:],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,52:57],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,25:40],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,17])
#y_actual = df1.iloc[:,25:29]

X_train,X_test,y_train,y_test = train_test_split(X_10,y_10)
X_train2,X_test2,y_train2,y_test2 = train_test_split(X2,y2)
X_train3,X_test3,y_train3,y_test3 = train_test_split(X3,y3)
X_train4,X_test4,y_train4,y_test4 = train_test_split(X4,y4)
X_train5,X_test5,y_train5,y_test5 = train_test_split(X5,y5)
X_train6,X_test6,y_train6,y_test6 = train_test_split(X6,y6)
X_train7,X_test7,y_train7,y_test7 = train_test_split(X7,y7)
X_train8,X_test8,y_train8,y_test8 = train_test_split(X8,y8)
X_train9,X_test9,y_train9,y_test9 = train_test_split(X9,y9)
X_train10,X_test10,y_train10,y_test10 = train_test_split(X10,y10)
X_train11,X_test11,y_train11,y_test11 = train_test_split(X11,y11)
X_train12,X_test12,y_train12,y_test12 = train_test_split(X12,y12)
X_train13,X_test13,y_train13,y_test13 = train_test_split(X13,y13)
X_train14,X_test14,y_train14,y_test14 = train_test_split(X14,y14)
X_train15,X_test15,y_train15,y_test15 = train_test_split(X15,y15)

In [71]:

#L1FO Neural Network decorrelated with 10 features


#Create the main Neural Network
model = Sequential()
model.add(Dense(200, activation = 'relu',input_shape=(10,)))
model.add(Dense(10,activation='relu'))
model.add(Dense(4,activation='sigmoid'))#Changed to sigmoid to allow 0 to 1

model.compile(loss='mse',optimizer='adam')

model.fit(X_train,y_train,batch_size=50,epochs=200,verbose=2,validation_data=(X_test,y_test),shuffle=False) #Changed shuffle to true
model.fit(X_train2,y_train2,batch_size=50,epochs=200,verbose=2,validation_data=(X_test2,y_test2),shuffle=False) #Changed shuffle to true
model.fit(X_train3,y_train3,batch_size=50,epochs=200,verbose=2,validation_data=(X_test3,y_test3),shuffle=False) #Changed shuffle to true
model.fit(X_train4,y_train4,batch_size=50,epochs=200,verbose=2,validation_data=(X_test4,y_test4),shuffle=False) #Changed shuffle to true
model.fit(X_train5,y_train5,batch_size=50,epochs=200,verbose=2,validation_data=(X_test5,y_test5),shuffle=False) #Changed shuffle to true
model.fit(X_train6,y_train6,batch_size=50,epochs=200,verbose=2,validation_data=(X_test6,y_test6),shuffle=False) #Changed shuffle to true
model.fit(X_train7,y_train7,batch_size=50,epochs=200,verbose=2,validation_data=(X_test7,y_test7),shuffle=False) #Changed shuffle to true
model.fit(X_train8,y_train8,batch_size=50,epochs=200,verbose=2,validation_data=(X_test8,y_test8),shuffle=False) #Changed shuffle to true
model.fit(X_train9,y_train9,batch_size=50,epochs=200,verbose=2,validation_data=(X_test9,y_test9),shuffle=False) #Changed shuffle to true
model.fit(X_train10,y_train10,batch_size=50,epochs=200,verbose=2,validation_data=(X_test10,y_test10),shuffle=False) #Changed shuffle to true
model.fit(X_train11,y_train11,batch_size=50,epochs=200,verbose=2,validation_data=(X_test11,y_test11),shuffle=False) #Changed shuffle to true
model.fit(X_train12,y_train12,batch_size=50,epochs=200,verbose=2,validation_data=(X_test12,y_test12),shuffle=False) #Changed shuffle to true
model.fit(X_train13,y_train13,batch_size=50,epochs=200,verbose=2,validation_data=(X_test13,y_test13),shuffle=False) #Changed shuffle to true
model.fit(X_train14,y_train14,batch_size=50,epochs=200,verbose=2,validation_data=(X_test14,y_test14),shuffle=False) #Changed shuffle to true
model.fit(X_train15,y_train15,batch_size=50,epochs=200,verbose=2,validation_data=(X_test15,y_test15),shuffle=False) #Changed shuffle to true 


y_pred = model.predict(X_pred10)
result10 = mse(y_actual10,y_pred)



Train on 2139 samples, validate on 713 samples
Epoch 1/200
 - 12s - loss: 0.0841 - val_loss: 0.0828
Epoch 2/200
 - 1s - loss: 0.0792 - val_loss: 0.0776
Epoch 3/200
 - 1s - loss: 0.0734 - val_loss: 0.0714
Epoch 4/200
 - 0s - loss: 0.0650 - val_loss: 0.0570
Epoch 5/200
 - 1s - loss: 0.0452 - val_loss: 0.0379
Epoch 6/200
 - 1s - loss: 0.0331 - val_loss: 0.0333
Epoch 7/200
 - 1s - loss: 0.0304 - val_loss: 0.0317
Epoch 8/200
 - 1s - loss: 0.0288 - val_loss: 0.0303
Epoch 9/200
 - 0s - loss: 0.0274 - val_loss: 0.0291
Epoch 10/200
 - 0s - loss: 0.0262 - val_loss: 0.0280
Epoch 11/200
 - 0s - loss: 0.0250 - val_loss: 0.0269
Epoch 12/200
 - 0s - loss: 0.0240 - val_loss: 0.0260
Epoch 13/200
 - 0s - loss: 0.0231 - val_loss: 0.0251
Epoch 14/200
 - 0s - loss: 0.0222 - val_loss: 0.0243
Epoch 15/200
 - 0s - loss: 0.0214 - val_loss: 0.0235
Epoch 16/200
 - 0s - loss: 0.0204 - val_loss: 0.0224
Epoch 17/200
 - 0s - loss: 0.0187 - val_loss: 0.0202
Epoch 18/200
 - 0s - loss: 0.0169 - val_loss: 0.0184
Epoch 1

Epoch 154/200
 - 0s - loss: 0.0062 - val_loss: 0.0079
Epoch 155/200
 - 0s - loss: 0.0062 - val_loss: 0.0079
Epoch 156/200
 - 0s - loss: 0.0062 - val_loss: 0.0078
Epoch 157/200
 - 0s - loss: 0.0062 - val_loss: 0.0078
Epoch 158/200
 - 0s - loss: 0.0062 - val_loss: 0.0078
Epoch 159/200
 - 0s - loss: 0.0062 - val_loss: 0.0078
Epoch 160/200
 - 0s - loss: 0.0061 - val_loss: 0.0078
Epoch 161/200
 - 0s - loss: 0.0061 - val_loss: 0.0078
Epoch 162/200
 - 0s - loss: 0.0061 - val_loss: 0.0078
Epoch 163/200
 - 0s - loss: 0.0061 - val_loss: 0.0078
Epoch 164/200
 - 0s - loss: 0.0061 - val_loss: 0.0077
Epoch 165/200
 - 0s - loss: 0.0061 - val_loss: 0.0077
Epoch 166/200
 - 0s - loss: 0.0060 - val_loss: 0.0077
Epoch 167/200
 - 0s - loss: 0.0060 - val_loss: 0.0077
Epoch 168/200
 - 0s - loss: 0.0060 - val_loss: 0.0077
Epoch 169/200
 - 0s - loss: 0.0060 - val_loss: 0.0077
Epoch 170/200
 - 0s - loss: 0.0060 - val_loss: 0.0077
Epoch 171/200
 - 0s - loss: 0.0060 - val_loss: 0.0077
Epoch 172/200
 - 0s - loss: 

Epoch 107/200
 - 0s - loss: 0.0055 - val_loss: 0.0072
Epoch 108/200
 - 0s - loss: 0.0055 - val_loss: 0.0072
Epoch 109/200
 - 0s - loss: 0.0055 - val_loss: 0.0072
Epoch 110/200
 - 0s - loss: 0.0054 - val_loss: 0.0071
Epoch 111/200
 - 0s - loss: 0.0054 - val_loss: 0.0071
Epoch 112/200
 - 0s - loss: 0.0054 - val_loss: 0.0071
Epoch 113/200
 - 0s - loss: 0.0054 - val_loss: 0.0071
Epoch 114/200
 - 0s - loss: 0.0053 - val_loss: 0.0070
Epoch 115/200
 - 0s - loss: 0.0053 - val_loss: 0.0070
Epoch 116/200
 - 0s - loss: 0.0053 - val_loss: 0.0070
Epoch 117/200
 - 0s - loss: 0.0053 - val_loss: 0.0070
Epoch 118/200
 - 0s - loss: 0.0052 - val_loss: 0.0069
Epoch 119/200
 - 0s - loss: 0.0052 - val_loss: 0.0069
Epoch 120/200
 - 0s - loss: 0.0052 - val_loss: 0.0069
Epoch 121/200
 - 0s - loss: 0.0052 - val_loss: 0.0069
Epoch 122/200
 - 0s - loss: 0.0052 - val_loss: 0.0069
Epoch 123/200
 - 0s - loss: 0.0051 - val_loss: 0.0068
Epoch 124/200
 - 0s - loss: 0.0051 - val_loss: 0.0068
Epoch 125/200
 - 0s - loss: 

 - 1s - loss: 0.0039 - val_loss: 0.0043
Epoch 60/200
 - 1s - loss: 0.0039 - val_loss: 0.0043
Epoch 61/200
 - 1s - loss: 0.0038 - val_loss: 0.0043
Epoch 62/200
 - 1s - loss: 0.0038 - val_loss: 0.0042
Epoch 63/200
 - 1s - loss: 0.0038 - val_loss: 0.0042
Epoch 64/200
 - 1s - loss: 0.0038 - val_loss: 0.0042
Epoch 65/200
 - 1s - loss: 0.0038 - val_loss: 0.0042
Epoch 66/200
 - 1s - loss: 0.0037 - val_loss: 0.0042
Epoch 67/200
 - 1s - loss: 0.0037 - val_loss: 0.0042
Epoch 68/200
 - 1s - loss: 0.0037 - val_loss: 0.0042
Epoch 69/200
 - 1s - loss: 0.0037 - val_loss: 0.0041
Epoch 70/200
 - 1s - loss: 0.0037 - val_loss: 0.0041
Epoch 71/200
 - 1s - loss: 0.0036 - val_loss: 0.0041
Epoch 72/200
 - 1s - loss: 0.0036 - val_loss: 0.0041
Epoch 73/200
 - 1s - loss: 0.0036 - val_loss: 0.0041
Epoch 74/200
 - 1s - loss: 0.0036 - val_loss: 0.0040
Epoch 75/200
 - 1s - loss: 0.0036 - val_loss: 0.0040
Epoch 76/200
 - 1s - loss: 0.0035 - val_loss: 0.0040
Epoch 77/200
 - 1s - loss: 0.0035 - val_loss: 0.0040
Epoch 

 - 0s - loss: 0.0102 - val_loss: 0.0103
Epoch 12/200
 - 0s - loss: 0.0099 - val_loss: 0.0100
Epoch 13/200
 - 0s - loss: 0.0096 - val_loss: 0.0098
Epoch 14/200
 - 0s - loss: 0.0094 - val_loss: 0.0096
Epoch 15/200
 - 0s - loss: 0.0092 - val_loss: 0.0094
Epoch 16/200
 - 0s - loss: 0.0090 - val_loss: 0.0092
Epoch 17/200
 - 0s - loss: 0.0088 - val_loss: 0.0090
Epoch 18/200
 - 0s - loss: 0.0086 - val_loss: 0.0089
Epoch 19/200
 - 0s - loss: 0.0085 - val_loss: 0.0087
Epoch 20/200
 - 0s - loss: 0.0083 - val_loss: 0.0086
Epoch 21/200
 - 0s - loss: 0.0082 - val_loss: 0.0085
Epoch 22/200
 - 0s - loss: 0.0080 - val_loss: 0.0083
Epoch 23/200
 - 0s - loss: 0.0078 - val_loss: 0.0082
Epoch 24/200
 - 0s - loss: 0.0077 - val_loss: 0.0081
Epoch 25/200
 - 0s - loss: 0.0076 - val_loss: 0.0080
Epoch 26/200
 - 0s - loss: 0.0075 - val_loss: 0.0079
Epoch 27/200
 - 0s - loss: 0.0074 - val_loss: 0.0078
Epoch 28/200
 - 0s - loss: 0.0073 - val_loss: 0.0077
Epoch 29/200
 - 0s - loss: 0.0072 - val_loss: 0.0077
Epoch 

...

Epoch 93/200
 - 1s - loss: 0.0025 - val_loss: 0.0028
Epoch 94/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 95/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 96/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 97/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 98/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 99/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 100/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 101/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 102/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 103/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 104/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 105/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 106/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 107/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 108/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 109/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 110/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 111/200
 - 1s - loss: 0.0023 

Epoch 45/200
 - 1s - loss: 0.0040 - val_loss: 0.0047
Epoch 46/200
 - 1s - loss: 0.0040 - val_loss: 0.0047
Epoch 47/200
 - 1s - loss: 0.0039 - val_loss: 0.0046
Epoch 48/200
 - 1s - loss: 0.0039 - val_loss: 0.0046
Epoch 49/200
 - 1s - loss: 0.0039 - val_loss: 0.0046
Epoch 50/200
 - 1s - loss: 0.0039 - val_loss: 0.0046
Epoch 51/200
 - 1s - loss: 0.0039 - val_loss: 0.0046
Epoch 52/200
 - 1s - loss: 0.0038 - val_loss: 0.0045
Epoch 53/200
 - 1s - loss: 0.0038 - val_loss: 0.0046
Epoch 54/200
 - 1s - loss: 0.0038 - val_loss: 0.0045
Epoch 55/200
 - 1s - loss: 0.0038 - val_loss: 0.0045
Epoch 56/200
 - 1s - loss: 0.0038 - val_loss: 0.0045
Epoch 57/200
 - 1s - loss: 0.0038 - val_loss: 0.0045
Epoch 58/200
 - 1s - loss: 0.0038 - val_loss: 0.0045
Epoch 59/200
 - 1s - loss: 0.0037 - val_loss: 0.0045
Epoch 60/200
 - 1s - loss: 0.0037 - val_loss: 0.0045
Epoch 61/200
 - 1s - loss: 0.0037 - val_loss: 0.0044
Epoch 62/200
 - 1s - loss: 0.0037 - val_loss: 0.0044
Epoch 63/200
 - 1s - loss: 0.0037 - val_loss: 

Epoch 198/200
 - 1s - loss: 0.0029 - val_loss: 0.0038
Epoch 199/200
 - 1s - loss: 0.0029 - val_loss: 0.0038
Epoch 200/200
 - 1s - loss: 0.0029 - val_loss: 0.0038
Train on 3639 samples, validate on 1213 samples
Epoch 1/200
 - 1s - loss: 0.0238 - val_loss: 0.0110
Epoch 2/200
 - 1s - loss: 0.0106 - val_loss: 0.0095
Epoch 3/200
 - 1s - loss: 0.0092 - val_loss: 0.0085
Epoch 4/200
 - 1s - loss: 0.0084 - val_loss: 0.0080
Epoch 5/200
 - 1s - loss: 0.0079 - val_loss: 0.0077
Epoch 6/200
 - 1s - loss: 0.0073 - val_loss: 0.0072
Epoch 7/200
 - 1s - loss: 0.0069 - val_loss: 0.0069
Epoch 8/200
 - 1s - loss: 0.0067 - val_loss: 0.0068
Epoch 9/200
 - 1s - loss: 0.0065 - val_loss: 0.0066
Epoch 10/200
 - 1s - loss: 0.0063 - val_loss: 0.0064
Epoch 11/200
 - 1s - loss: 0.0061 - val_loss: 0.0063
Epoch 12/200
 - 1s - loss: 0.0060 - val_loss: 0.0061
Epoch 13/200
 - 1s - loss: 0.0059 - val_loss: 0.0060
Epoch 14/200
 - 1s - loss: 0.0058 - val_loss: 0.0059
Epoch 15/200
 - 1s - loss: 0.0057 - val_loss: 0.0058
Epoc

Epoch 151/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 152/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 153/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 154/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 155/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 156/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 157/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 158/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 159/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 160/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 161/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 162/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 163/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 164/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 165/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 166/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 167/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 168/200
 - 1s - loss: 0.0032 - val_loss: 0.0036
Epoch 169/200
 - 1s - loss: 

Epoch 104/200
 - 1s - loss: 0.0037 - val_loss: 0.0050
Epoch 105/200
 - 1s - loss: 0.0037 - val_loss: 0.0050
Epoch 106/200
 - 1s - loss: 0.0037 - val_loss: 0.0049
Epoch 107/200
 - 1s - loss: 0.0037 - val_loss: 0.0049
Epoch 108/200
 - 1s - loss: 0.0037 - val_loss: 0.0049
Epoch 109/200
 - 1s - loss: 0.0037 - val_loss: 0.0049
Epoch 110/200
 - 1s - loss: 0.0037 - val_loss: 0.0049
Epoch 111/200
 - 1s - loss: 0.0036 - val_loss: 0.0049
Epoch 112/200
 - 1s - loss: 0.0036 - val_loss: 0.0049
Epoch 113/200
 - 1s - loss: 0.0036 - val_loss: 0.0049
Epoch 114/200
 - 1s - loss: 0.0036 - val_loss: 0.0049
Epoch 115/200
 - 1s - loss: 0.0036 - val_loss: 0.0049
Epoch 116/200
 - 1s - loss: 0.0036 - val_loss: 0.0049
Epoch 117/200
 - 1s - loss: 0.0036 - val_loss: 0.0049
Epoch 118/200
 - 1s - loss: 0.0036 - val_loss: 0.0049
Epoch 119/200
 - 1s - loss: 0.0036 - val_loss: 0.0049
Epoch 120/200
 - 1s - loss: 0.0036 - val_loss: 0.0049
Epoch 121/200
 - 1s - loss: 0.0036 - val_loss: 0.0049
Epoch 122/200
 - 1s - loss: 

In [72]:
X_20 = df[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y_20 = df[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X20=MinMaxScaler()
scaler_X20 = scaler_X20.fit_transform(X_20)
X_20 = pd.DataFrame(scaler_X20)
scaler_y20=MinMaxScaler()
scaler_y20 = scaler_y20.fit_transform(y_20)
y_20 = pd.DataFrame(scaler_y20)

X_pred20 = df1[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y_actual20 = df1[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X_pred20)
X_pred20 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y_actual20)
y_actual20 = pd.DataFrame(scaler_y)

X2 = df2[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y2 = df2[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X2)
X2 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y2)
y2 = pd.DataFrame(scaler_y)

X3 = df3[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y3 = df3[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X3)
X3 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y3)
y3 = pd.DataFrame(scaler_y)

X4 = df4[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y4 = df4[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X4)
X4 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y4)
y4 = pd.DataFrame(scaler_y)

X5 = df5[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y5 = df5[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X5)
X5 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y5)
y5 = pd.DataFrame(scaler_y)

X6 = df6[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y6 = df6[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X6)
X6 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y6)
y6 = pd.DataFrame(scaler_y)

X7 = df7[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y7 = df7[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X7)
X7 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y7)
y7 = pd.DataFrame(scaler_y)

X8 = df8[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y8 = df8[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X8)
X8 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y8)
y8 = pd.DataFrame(scaler_y)

X9 = df9[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y9 = df9[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X9)
X9 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y9)
y9 = pd.DataFrame(scaler_y)

X10 = df10[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y10 = df10[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X10)
X10 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y10)
y10 = pd.DataFrame(scaler_y)

X11 = df11[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y11 = df11[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X11)
X11 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y11)
y11 = pd.DataFrame(scaler_y)

X12 = df12[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y12 = df12[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X12)
X12 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y12)
y12 = pd.DataFrame(scaler_y)

X13 = df13[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y13 = df13[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X13)
X13 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y13)
y13 = pd.DataFrame(scaler_y)

X14 = df14[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y14 = df14[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X14)
X14 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y14)
y14 = pd.DataFrame(scaler_y)

X15 = df15[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y15 = df15[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X15)
X15 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y15)
y15 = pd.DataFrame(scaler_y)

X_pred = df1[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4']]
y_actual = df1[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X_pred)
X_pred = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y_actual)
y_actual = pd.DataFrame(scaler_y)

#X_pred = df1.drop(df1.ix[:,69:],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,52:57],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,25:40],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,17])
#y_actual = df1.iloc[:,25:29]

X_train,X_test,y_train,y_test = train_test_split(X_20,y_20)
X_train2,X_test2,y_train2,y_test2 = train_test_split(X2,y2)
X_train3,X_test3,y_train3,y_test3 = train_test_split(X3,y3)
X_train4,X_test4,y_train4,y_test4 = train_test_split(X4,y4)
X_train5,X_test5,y_train5,y_test5 = train_test_split(X5,y5)
X_train6,X_test6,y_train6,y_test6 = train_test_split(X6,y6)
X_train7,X_test7,y_train7,y_test7 = train_test_split(X7,y7)
X_train8,X_test8,y_train8,y_test8 = train_test_split(X8,y8)
X_train9,X_test9,y_train9,y_test9 = train_test_split(X9,y9)
X_train10,X_test10,y_train10,y_test10 = train_test_split(X10,y10)
X_train11,X_test11,y_train11,y_test11 = train_test_split(X11,y11)
X_train12,X_test12,y_train12,y_test12 = train_test_split(X12,y12)
X_train13,X_test13,y_train13,y_test13 = train_test_split(X13,y13)
X_train14,X_test14,y_train14,y_test14 = train_test_split(X14,y14)
X_train15,X_test15,y_train15,y_test15 = train_test_split(X15,y15)

#L1FO Neural Network decorrelated with 20 features


#Create the main Neural Network
model = Sequential()
model.add(Dense(10, activation = 'relu',input_shape=(20,)))
model.add(Dense(25,activation='relu'))
model.add(Dense(4,activation='sigmoid'))#Changed to sigmoid to allow 0 to 1

model.compile(loss='mse',optimizer='adam')

model.fit(X_train,y_train,batch_size=50,epochs=200,verbose=2,validation_data=(X_test,y_test),shuffle=False) #Changed shuffle to true
model.fit(X_train2,y_train2,batch_size=50,epochs=200,verbose=2,validation_data=(X_test2,y_test2),shuffle=False) #Changed shuffle to true
model.fit(X_train3,y_train3,batch_size=50,epochs=200,verbose=2,validation_data=(X_test3,y_test3),shuffle=False) #Changed shuffle to true
model.fit(X_train4,y_train4,batch_size=50,epochs=200,verbose=2,validation_data=(X_test4,y_test4),shuffle=False) #Changed shuffle to true
model.fit(X_train5,y_train5,batch_size=50,epochs=200,verbose=2,validation_data=(X_test5,y_test5),shuffle=False) #Changed shuffle to true
model.fit(X_train6,y_train6,batch_size=50,epochs=200,verbose=2,validation_data=(X_test6,y_test6),shuffle=False) #Changed shuffle to true
model.fit(X_train7,y_train7,batch_size=50,epochs=200,verbose=2,validation_data=(X_test7,y_test7),shuffle=False) #Changed shuffle to true
model.fit(X_train8,y_train8,batch_size=50,epochs=200,verbose=2,validation_data=(X_test8,y_test8),shuffle=False) #Changed shuffle to true
model.fit(X_train9,y_train9,batch_size=50,epochs=200,verbose=2,validation_data=(X_test9,y_test9),shuffle=False) #Changed shuffle to true
model.fit(X_train10,y_train10,batch_size=50,epochs=200,verbose=2,validation_data=(X_test10,y_test10),shuffle=False) #Changed shuffle to true
model.fit(X_train11,y_train11,batch_size=50,epochs=200,verbose=2,validation_data=(X_test11,y_test11),shuffle=False) #Changed shuffle to true
model.fit(X_train12,y_train12,batch_size=50,epochs=200,verbose=2,validation_data=(X_test12,y_test12),shuffle=False) #Changed shuffle to true
model.fit(X_train13,y_train13,batch_size=50,epochs=200,verbose=2,validation_data=(X_test13,y_test13),shuffle=False) #Changed shuffle to true
model.fit(X_train14,y_train14,batch_size=50,epochs=200,verbose=2,validation_data=(X_test14,y_test14),shuffle=False) #Changed shuffle to true
model.fit(X_train15,y_train15,batch_size=50,epochs=200,verbose=2,validation_data=(X_test15,y_test15),shuffle=False) #Changed shuffle to true 


y_pred = model.predict(X_pred20)
result20 = mse(y_actual20,y_pred)


C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\an

Train on 2139 samples, validate on 713 samples
Epoch 1/200
 - 10s - loss: 0.0780 - val_loss: 0.0660
Epoch 2/200
 - 0s - loss: 0.0557 - val_loss: 0.0442
Epoch 3/200
 - 0s - loss: 0.0370 - val_loss: 0.0317
Epoch 4/200
 - 0s - loss: 0.0287 - val_loss: 0.0266
Epoch 5/200
 - 0s - loss: 0.0244 - val_loss: 0.0228
Epoch 6/200
 - 0s - loss: 0.0210 - val_loss: 0.0196
Epoch 7/200
 - 0s - loss: 0.0182 - val_loss: 0.0169
Epoch 8/200
 - 0s - loss: 0.0158 - val_loss: 0.0147
Epoch 9/200
 - 0s - loss: 0.0139 - val_loss: 0.0130
Epoch 10/200
 - 0s - loss: 0.0125 - val_loss: 0.0116
Epoch 11/200
 - 0s - loss: 0.0114 - val_loss: 0.0107
Epoch 12/200
 - 0s - loss: 0.0106 - val_loss: 0.0100
Epoch 13/200
 - 0s - loss: 0.0100 - val_loss: 0.0094
Epoch 14/200
 - 0s - loss: 0.0095 - val_loss: 0.0090
Epoch 15/200
 - 0s - loss: 0.0091 - val_loss: 0.0087
Epoch 16/200
 - 0s - loss: 0.0088 - val_loss: 0.0084
Epoch 17/200
 - 0s - loss: 0.0085 - val_loss: 0.0081
Epoch 18/200
 - 0s - loss: 0.0083 - val_loss: 0.0079
Epoch 1

Epoch 154/200
 - 1s - loss: 0.0036 - val_loss: 0.0035
Epoch 155/200
 - 0s - loss: 0.0036 - val_loss: 0.0035
Epoch 156/200
 - 0s - loss: 0.0036 - val_loss: 0.0034
Epoch 157/200
 - 1s - loss: 0.0036 - val_loss: 0.0034
Epoch 158/200
 - 0s - loss: 0.0036 - val_loss: 0.0034
Epoch 159/200
 - 0s - loss: 0.0036 - val_loss: 0.0034
Epoch 160/200
 - 0s - loss: 0.0036 - val_loss: 0.0034
Epoch 161/200
 - 0s - loss: 0.0036 - val_loss: 0.0034
Epoch 162/200
 - 0s - loss: 0.0035 - val_loss: 0.0034
Epoch 163/200
 - 0s - loss: 0.0035 - val_loss: 0.0034
Epoch 164/200
 - 1s - loss: 0.0035 - val_loss: 0.0034
Epoch 165/200
 - 0s - loss: 0.0035 - val_loss: 0.0034
Epoch 166/200
 - 0s - loss: 0.0035 - val_loss: 0.0034
Epoch 167/200
 - 0s - loss: 0.0035 - val_loss: 0.0034
Epoch 168/200
 - 0s - loss: 0.0035 - val_loss: 0.0034
Epoch 169/200
 - 0s - loss: 0.0035 - val_loss: 0.0034
Epoch 170/200
 - 0s - loss: 0.0035 - val_loss: 0.0034
Epoch 171/200
 - 0s - loss: 0.0035 - val_loss: 0.0034
Epoch 172/200
 - 0s - loss: 

Epoch 107/200
 - 0s - loss: 0.0067 - val_loss: 0.0067
Epoch 108/200
 - 0s - loss: 0.0067 - val_loss: 0.0067
Epoch 109/200
 - 0s - loss: 0.0067 - val_loss: 0.0066
Epoch 110/200
 - 0s - loss: 0.0067 - val_loss: 0.0066
Epoch 111/200
 - 0s - loss: 0.0066 - val_loss: 0.0066
Epoch 112/200
 - 0s - loss: 0.0066 - val_loss: 0.0066
Epoch 113/200
 - 0s - loss: 0.0066 - val_loss: 0.0065
Epoch 114/200
 - 0s - loss: 0.0065 - val_loss: 0.0065
Epoch 115/200
 - 0s - loss: 0.0065 - val_loss: 0.0065
Epoch 116/200
 - 0s - loss: 0.0065 - val_loss: 0.0065
Epoch 117/200
 - 0s - loss: 0.0065 - val_loss: 0.0064
Epoch 118/200
 - 0s - loss: 0.0064 - val_loss: 0.0064
Epoch 119/200
 - 0s - loss: 0.0064 - val_loss: 0.0064
Epoch 120/200
 - 0s - loss: 0.0064 - val_loss: 0.0064
Epoch 121/200
 - 0s - loss: 0.0063 - val_loss: 0.0063
Epoch 122/200
 - 0s - loss: 0.0063 - val_loss: 0.0063
Epoch 123/200
 - 0s - loss: 0.0063 - val_loss: 0.0063
Epoch 124/200
 - 0s - loss: 0.0063 - val_loss: 0.0063
Epoch 125/200
 - 0s - loss: 

 - 1s - loss: 0.0027 - val_loss: 0.0030
Epoch 60/200
 - 1s - loss: 0.0026 - val_loss: 0.0030
Epoch 61/200
 - 1s - loss: 0.0026 - val_loss: 0.0029
Epoch 62/200
 - 1s - loss: 0.0026 - val_loss: 0.0029
Epoch 63/200
 - 1s - loss: 0.0026 - val_loss: 0.0029
Epoch 64/200
 - 1s - loss: 0.0026 - val_loss: 0.0029
Epoch 65/200
 - 1s - loss: 0.0026 - val_loss: 0.0029
Epoch 66/200
 - 1s - loss: 0.0026 - val_loss: 0.0029
Epoch 67/200
 - 1s - loss: 0.0025 - val_loss: 0.0029
Epoch 68/200
 - 1s - loss: 0.0025 - val_loss: 0.0029
Epoch 69/200
 - 1s - loss: 0.0025 - val_loss: 0.0028
Epoch 70/200
 - 1s - loss: 0.0025 - val_loss: 0.0028
Epoch 71/200
 - 1s - loss: 0.0025 - val_loss: 0.0028
Epoch 72/200
 - 1s - loss: 0.0025 - val_loss: 0.0028
Epoch 73/200
 - 1s - loss: 0.0025 - val_loss: 0.0028
Epoch 74/200
 - 1s - loss: 0.0025 - val_loss: 0.0028
Epoch 75/200
 - 1s - loss: 0.0025 - val_loss: 0.0028
Epoch 76/200
 - 1s - loss: 0.0025 - val_loss: 0.0028
Epoch 77/200
 - 1s - loss: 0.0024 - val_loss: 0.0028
Epoch 

...

Epoch 93/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 94/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 95/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 96/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 97/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 98/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 99/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 100/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 101/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 102/200
 - 2s - loss: 0.0021 - val_loss: 0.0022
Epoch 103/200
 - 2s - loss: 0.0021 - val_loss: 0.0022
Epoch 104/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 105/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 106/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 107/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 108/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 109/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 110/200
 - 1s - loss: 0.0021 - val_loss: 0.0022
Epoch 111/200
 - 1s - loss: 0.0020 

Epoch 45/200
 - 1s - loss: 0.0039 - val_loss: 0.0043
Epoch 46/200
 - 1s - loss: 0.0038 - val_loss: 0.0042
Epoch 47/200
 - 1s - loss: 0.0038 - val_loss: 0.0042
Epoch 48/200
 - 1s - loss: 0.0038 - val_loss: 0.0042
Epoch 49/200
 - 1s - loss: 0.0038 - val_loss: 0.0042
Epoch 50/200
 - 1s - loss: 0.0038 - val_loss: 0.0042
Epoch 51/200
 - 1s - loss: 0.0038 - val_loss: 0.0041
Epoch 52/200
 - 1s - loss: 0.0037 - val_loss: 0.0041
Epoch 53/200
 - 1s - loss: 0.0037 - val_loss: 0.0041
Epoch 54/200
 - 1s - loss: 0.0037 - val_loss: 0.0041
Epoch 55/200
 - 1s - loss: 0.0037 - val_loss: 0.0041
Epoch 56/200
 - 1s - loss: 0.0037 - val_loss: 0.0041
Epoch 57/200
 - 1s - loss: 0.0037 - val_loss: 0.0040
Epoch 58/200
 - 1s - loss: 0.0036 - val_loss: 0.0040
Epoch 59/200
 - 1s - loss: 0.0036 - val_loss: 0.0040
Epoch 60/200
 - 1s - loss: 0.0036 - val_loss: 0.0040
Epoch 61/200
 - 1s - loss: 0.0036 - val_loss: 0.0040
Epoch 62/200
 - 1s - loss: 0.0036 - val_loss: 0.0040
Epoch 63/200
 - 1s - loss: 0.0036 - val_loss: 

Epoch 198/200
 - 1s - loss: 0.0028 - val_loss: 0.0032
Epoch 199/200
 - 1s - loss: 0.0028 - val_loss: 0.0032
Epoch 200/200
 - 1s - loss: 0.0028 - val_loss: 0.0032
Train on 3639 samples, validate on 1213 samples
Epoch 1/200
 - 1s - loss: 0.0140 - val_loss: 0.0081
Epoch 2/200
 - 1s - loss: 0.0074 - val_loss: 0.0069
Epoch 3/200
 - 1s - loss: 0.0067 - val_loss: 0.0065
Epoch 4/200
 - 1s - loss: 0.0064 - val_loss: 0.0062
Epoch 5/200
 - 1s - loss: 0.0061 - val_loss: 0.0060
Epoch 6/200
 - 1s - loss: 0.0059 - val_loss: 0.0058
Epoch 7/200
 - 1s - loss: 0.0058 - val_loss: 0.0057
Epoch 8/200
 - 1s - loss: 0.0056 - val_loss: 0.0055
Epoch 9/200
 - 1s - loss: 0.0055 - val_loss: 0.0054
Epoch 10/200
 - 1s - loss: 0.0054 - val_loss: 0.0053
Epoch 11/200
 - 1s - loss: 0.0053 - val_loss: 0.0053
Epoch 12/200
 - 1s - loss: 0.0053 - val_loss: 0.0052
Epoch 13/200
 - 1s - loss: 0.0052 - val_loss: 0.0051
Epoch 14/200
 - 1s - loss: 0.0051 - val_loss: 0.0051
Epoch 15/200
 - 1s - loss: 0.0051 - val_loss: 0.0050
Epoc

Epoch 151/200
 - 1s - loss: 0.0027 - val_loss: 0.0029
Epoch 152/200
 - 1s - loss: 0.0027 - val_loss: 0.0029
Epoch 153/200
 - 1s - loss: 0.0027 - val_loss: 0.0029
Epoch 154/200
 - 1s - loss: 0.0027 - val_loss: 0.0029
Epoch 155/200
 - 1s - loss: 0.0027 - val_loss: 0.0029
Epoch 156/200
 - 1s - loss: 0.0027 - val_loss: 0.0029
Epoch 157/200
 - 1s - loss: 0.0027 - val_loss: 0.0028
Epoch 158/200
 - 1s - loss: 0.0026 - val_loss: 0.0028
Epoch 159/200
 - 1s - loss: 0.0026 - val_loss: 0.0028
Epoch 160/200
 - 1s - loss: 0.0026 - val_loss: 0.0028
Epoch 161/200
 - 1s - loss: 0.0026 - val_loss: 0.0028
Epoch 162/200
 - 1s - loss: 0.0026 - val_loss: 0.0028
Epoch 163/200
 - 1s - loss: 0.0026 - val_loss: 0.0028
Epoch 164/200
 - 1s - loss: 0.0026 - val_loss: 0.0028
Epoch 165/200
 - 1s - loss: 0.0026 - val_loss: 0.0028
Epoch 166/200
 - 1s - loss: 0.0026 - val_loss: 0.0028
Epoch 167/200
 - 1s - loss: 0.0026 - val_loss: 0.0028
Epoch 168/200
 - 1s - loss: 0.0026 - val_loss: 0.0028
Epoch 169/200
 - 1s - loss: 

Epoch 104/200
 - 1s - loss: 0.0035 - val_loss: 0.0037
Epoch 105/200
 - 1s - loss: 0.0035 - val_loss: 0.0037
Epoch 106/200
 - 1s - loss: 0.0034 - val_loss: 0.0037
Epoch 107/200
 - 1s - loss: 0.0034 - val_loss: 0.0037
Epoch 108/200
 - 1s - loss: 0.0034 - val_loss: 0.0037
Epoch 109/200
 - 1s - loss: 0.0034 - val_loss: 0.0037
Epoch 110/200
 - 1s - loss: 0.0034 - val_loss: 0.0037
Epoch 111/200
 - 1s - loss: 0.0034 - val_loss: 0.0037
Epoch 112/200
 - 1s - loss: 0.0034 - val_loss: 0.0036
Epoch 113/200
 - 1s - loss: 0.0034 - val_loss: 0.0036
Epoch 114/200
 - 1s - loss: 0.0034 - val_loss: 0.0036
Epoch 115/200
 - 1s - loss: 0.0034 - val_loss: 0.0036
Epoch 116/200
 - 1s - loss: 0.0034 - val_loss: 0.0036
Epoch 117/200
 - 1s - loss: 0.0034 - val_loss: 0.0036
Epoch 118/200
 - 1s - loss: 0.0034 - val_loss: 0.0036
Epoch 119/200
 - 1s - loss: 0.0033 - val_loss: 0.0036
Epoch 120/200
 - 1s - loss: 0.0033 - val_loss: 0.0036
Epoch 121/200
 - 1s - loss: 0.0033 - val_loss: 0.0036
Epoch 122/200
 - 1s - loss: 

In [73]:
X_30 = df[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y_30 = df[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X30=MinMaxScaler()
scaler_X30 = scaler_X30.fit_transform(X_30)
X_30 = pd.DataFrame(scaler_X30)
scaler_y30=MinMaxScaler()
scaler_y30 = scaler_y30.fit_transform(y_30)
y_30 = pd.DataFrame(scaler_y30)

X_pred30 = df1[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y_actual30 = df1[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X_pred30)
X_pred30 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y_actual30)
y_actual30 = pd.DataFrame(scaler_y)


X2 = df2[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y2 = df2[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X2)
X2 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y2)
y2 = pd.DataFrame(scaler_y)

X3 = df3[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y3 = df3[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X3)
X3 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y3)
y3 = pd.DataFrame(scaler_y)

X4 = df4[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y4 = df4[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X4)
X4 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y4)
y4 = pd.DataFrame(scaler_y)

X5 = df5[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y5 = df5[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X5)
X5 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y5)
y5 = pd.DataFrame(scaler_y)

X6 = df6[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y6 = df6[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X6)
X6 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y6)
y6 = pd.DataFrame(scaler_y)

X7 = df7[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y7 = df7[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X7)
X7 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y7)
y7 = pd.DataFrame(scaler_y)

X8 = df8[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y8 = df8[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X8)
X8 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y8)
y8 = pd.DataFrame(scaler_y)

X9 = df9[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y9 = df9[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X9)
X9 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y9)
y9 = pd.DataFrame(scaler_y)

X10 = df10[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y10 = df10[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X10)
X10 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y10)
y10 = pd.DataFrame(scaler_y)

X11 = df11[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y11 = df11[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X11)
X11 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y11)
y11 = pd.DataFrame(scaler_y)

X12 = df12[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y12 = df12[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X12)
X12 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y12)
y12 = pd.DataFrame(scaler_y)

X13 = df13[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y13 = df13[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X13)
X13 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y13)
y13 = pd.DataFrame(scaler_y)

X14 = df14[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y14 = df14[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X14)
X14 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y14)
y14 = pd.DataFrame(scaler_y)

X15 = df15[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y15 = df15[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X15)
X15 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y15)
y15 = pd.DataFrame(scaler_y)

X_pred = df1[['DS_Boomwz','DS_Bktwx','DS_Dipperwz','DS_BucketCylinderPressureB','DS_Dipper_Force','DS_Dipperwx','DS_Fuel_Meter','DS_Slew_Force','DS_Dipper_xdd','DS_Bktwz','DS_Upperwz','DS_CylinderBucketLift','DS_Slew_Angle','DS_BucketCylinderPressureA','DS_Bucket_Angle','DS_DSD2','DS_Upperwy','DS_Bktwy','SoilTransferSensor2 MassTot','DS_DST4','DS_DipperCylinderPressureB','DS_DSD1','DS_Boomwy','DS_Bkt_zdd','DS_Upperwx','DS_CylinderBoomLiftR','DS_DSD3','DS_DSDT','DS_Boom_Angle','DS_Dipperwy']]
y_actual = df1[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X_pred)
X_pred = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y_actual)
y_actual = pd.DataFrame(scaler_y)

#X_pred = df1.drop(df1.ix[:,69:],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,52:57],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,25:40],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,17])
#y_actual = df1.iloc[:,25:29]

X_train,X_test,y_train,y_test = train_test_split(X_30,y_30)
X_train2,X_test2,y_train2,y_test2 = train_test_split(X2,y2)
X_train3,X_test3,y_train3,y_test3 = train_test_split(X3,y3)
X_train4,X_test4,y_train4,y_test4 = train_test_split(X4,y4)
X_train5,X_test5,y_train5,y_test5 = train_test_split(X5,y5)
X_train6,X_test6,y_train6,y_test6 = train_test_split(X6,y6)
X_train7,X_test7,y_train7,y_test7 = train_test_split(X7,y7)
X_train8,X_test8,y_train8,y_test8 = train_test_split(X8,y8)
X_train9,X_test9,y_train9,y_test9 = train_test_split(X9,y9)
X_train10,X_test10,y_train10,y_test10 = train_test_split(X10,y10)
X_train11,X_test11,y_train11,y_test11 = train_test_split(X11,y11)
X_train12,X_test12,y_train12,y_test12 = train_test_split(X12,y12)
X_train13,X_test13,y_train13,y_test13 = train_test_split(X13,y13)
X_train14,X_test14,y_train14,y_test14 = train_test_split(X14,y14)
X_train15,X_test15,y_train15,y_test15 = train_test_split(X15,y15)

#Create the main Neural Network
model = Sequential()
model.add(Dense(100, activation = 'relu',input_shape=(30,)))
model.add(Dense(25,activation='relu'))
model.add(Dense(4,activation='sigmoid'))#Changed to sigmoid to allow 0 to 1

model.compile(loss='mse',optimizer='adam')

model.fit(X_train,y_train,batch_size=50,epochs=200,verbose=2,validation_data=(X_test,y_test),shuffle=False) #Changed shuffle to true
model.fit(X_train2,y_train2,batch_size=50,epochs=200,verbose=2,validation_data=(X_test2,y_test2),shuffle=False) #Changed shuffle to true
model.fit(X_train3,y_train3,batch_size=50,epochs=200,verbose=2,validation_data=(X_test3,y_test3),shuffle=False) #Changed shuffle to true
model.fit(X_train4,y_train4,batch_size=50,epochs=200,verbose=2,validation_data=(X_test4,y_test4),shuffle=False) #Changed shuffle to true
model.fit(X_train5,y_train5,batch_size=50,epochs=200,verbose=2,validation_data=(X_test5,y_test5),shuffle=False) #Changed shuffle to true
model.fit(X_train6,y_train6,batch_size=50,epochs=200,verbose=2,validation_data=(X_test6,y_test6),shuffle=False) #Changed shuffle to true
model.fit(X_train7,y_train7,batch_size=50,epochs=200,verbose=2,validation_data=(X_test7,y_test7),shuffle=False) #Changed shuffle to true
model.fit(X_train8,y_train8,batch_size=50,epochs=200,verbose=2,validation_data=(X_test8,y_test8),shuffle=False) #Changed shuffle to true
model.fit(X_train9,y_train9,batch_size=50,epochs=200,verbose=2,validation_data=(X_test9,y_test9),shuffle=False) #Changed shuffle to true
model.fit(X_train10,y_train10,batch_size=50,epochs=200,verbose=2,validation_data=(X_test10,y_test10),shuffle=False) #Changed shuffle to true
model.fit(X_train11,y_train11,batch_size=50,epochs=200,verbose=2,validation_data=(X_test11,y_test11),shuffle=False) #Changed shuffle to true
model.fit(X_train12,y_train12,batch_size=50,epochs=200,verbose=2,validation_data=(X_test12,y_test12),shuffle=False) #Changed shuffle to true
model.fit(X_train13,y_train13,batch_size=50,epochs=200,verbose=2,validation_data=(X_test13,y_test13),shuffle=False) #Changed shuffle to true
model.fit(X_train14,y_train14,batch_size=50,epochs=200,verbose=2,validation_data=(X_test14,y_test14),shuffle=False) #Changed shuffle to true
model.fit(X_train15,y_train15,batch_size=50,epochs=200,verbose=2,validation_data=(X_test15,y_test15),shuffle=False) #Changed shuffle to true 

y_pred = model.predict(X_pred30)
result30 = mse(y_actual30,y_pred)


C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\an

Train on 2139 samples, validate on 713 samples
Epoch 1/200
 - 10s - loss: 0.0512 - val_loss: 0.0301
Epoch 2/200
 - 0s - loss: 0.0252 - val_loss: 0.0210
Epoch 3/200
 - 0s - loss: 0.0181 - val_loss: 0.0158
Epoch 4/200
 - 0s - loss: 0.0138 - val_loss: 0.0126
Epoch 5/200
 - 0s - loss: 0.0110 - val_loss: 0.0105
Epoch 6/200
 - 0s - loss: 0.0093 - val_loss: 0.0091
Epoch 7/200
 - 0s - loss: 0.0082 - val_loss: 0.0083
Epoch 8/200
 - 0s - loss: 0.0075 - val_loss: 0.0076
Epoch 9/200
 - 0s - loss: 0.0070 - val_loss: 0.0071
Epoch 10/200
 - 0s - loss: 0.0066 - val_loss: 0.0067
Epoch 11/200
 - 0s - loss: 0.0063 - val_loss: 0.0064
Epoch 12/200
 - 0s - loss: 0.0060 - val_loss: 0.0061
Epoch 13/200
 - 0s - loss: 0.0057 - val_loss: 0.0058
Epoch 14/200
 - 0s - loss: 0.0054 - val_loss: 0.0055
Epoch 15/200
 - 0s - loss: 0.0052 - val_loss: 0.0053
Epoch 16/200
 - 0s - loss: 0.0050 - val_loss: 0.0051
Epoch 17/200
 - 0s - loss: 0.0048 - val_loss: 0.0049
Epoch 18/200
 - 0s - loss: 0.0047 - val_loss: 0.0048
Epoch 1

Epoch 154/200
 - 0s - loss: 0.0014 - val_loss: 0.0016
Epoch 155/200
 - 0s - loss: 0.0014 - val_loss: 0.0016
Epoch 156/200
 - 0s - loss: 0.0014 - val_loss: 0.0016
Epoch 157/200
 - 0s - loss: 0.0014 - val_loss: 0.0016
Epoch 158/200
 - 0s - loss: 0.0014 - val_loss: 0.0016
Epoch 159/200
 - 0s - loss: 0.0014 - val_loss: 0.0016
Epoch 160/200
 - 0s - loss: 0.0013 - val_loss: 0.0016
Epoch 161/200
 - 0s - loss: 0.0013 - val_loss: 0.0016
Epoch 162/200
 - 0s - loss: 0.0013 - val_loss: 0.0016
Epoch 163/200
 - 0s - loss: 0.0013 - val_loss: 0.0016
Epoch 164/200
 - 0s - loss: 0.0013 - val_loss: 0.0016
Epoch 165/200
 - 0s - loss: 0.0013 - val_loss: 0.0016
Epoch 166/200
 - 0s - loss: 0.0013 - val_loss: 0.0016
Epoch 167/200
 - 0s - loss: 0.0013 - val_loss: 0.0016
Epoch 168/200
 - 0s - loss: 0.0013 - val_loss: 0.0016
Epoch 169/200
 - 0s - loss: 0.0013 - val_loss: 0.0016
Epoch 170/200
 - 0s - loss: 0.0013 - val_loss: 0.0016
Epoch 171/200
 - 0s - loss: 0.0013 - val_loss: 0.0016
Epoch 172/200
 - 0s - loss: 

Epoch 107/200
 - 0s - loss: 0.0031 - val_loss: 0.0043
Epoch 108/200
 - 0s - loss: 0.0031 - val_loss: 0.0043
Epoch 109/200
 - 0s - loss: 0.0030 - val_loss: 0.0043
Epoch 110/200
 - 0s - loss: 0.0030 - val_loss: 0.0042
Epoch 111/200
 - 0s - loss: 0.0030 - val_loss: 0.0042
Epoch 112/200
 - 0s - loss: 0.0030 - val_loss: 0.0042
Epoch 113/200
 - 0s - loss: 0.0030 - val_loss: 0.0042
Epoch 114/200
 - 0s - loss: 0.0030 - val_loss: 0.0042
Epoch 115/200
 - 0s - loss: 0.0030 - val_loss: 0.0042
Epoch 116/200
 - 0s - loss: 0.0030 - val_loss: 0.0042
Epoch 117/200
 - 0s - loss: 0.0029 - val_loss: 0.0041
Epoch 118/200
 - 0s - loss: 0.0029 - val_loss: 0.0041
Epoch 119/200
 - 0s - loss: 0.0029 - val_loss: 0.0041
Epoch 120/200
 - 0s - loss: 0.0029 - val_loss: 0.0041
Epoch 121/200
 - 0s - loss: 0.0029 - val_loss: 0.0041
Epoch 122/200
 - 0s - loss: 0.0029 - val_loss: 0.0041
Epoch 123/200
 - 0s - loss: 0.0029 - val_loss: 0.0040
Epoch 124/200
 - 0s - loss: 0.0029 - val_loss: 0.0041
Epoch 125/200
 - 0s - loss: 

 - 1s - loss: 0.0010 - val_loss: 0.0012
Epoch 60/200
 - 1s - loss: 0.0010 - val_loss: 0.0012
Epoch 61/200
 - 1s - loss: 0.0010 - val_loss: 0.0012
Epoch 62/200
 - 1s - loss: 9.9098e-04 - val_loss: 0.0012
Epoch 63/200
 - 1s - loss: 9.8771e-04 - val_loss: 0.0011
Epoch 64/200
 - 1s - loss: 9.8326e-04 - val_loss: 0.0011
Epoch 65/200
 - 1s - loss: 9.7165e-04 - val_loss: 0.0011
Epoch 66/200
 - 1s - loss: 9.7336e-04 - val_loss: 0.0011
Epoch 67/200
 - 1s - loss: 9.6182e-04 - val_loss: 0.0011
Epoch 68/200
 - 1s - loss: 9.6038e-04 - val_loss: 0.0011
Epoch 69/200
 - 1s - loss: 9.5083e-04 - val_loss: 0.0011
Epoch 70/200
 - 1s - loss: 9.4770e-04 - val_loss: 0.0011
Epoch 71/200
 - 1s - loss: 9.4130e-04 - val_loss: 0.0011
Epoch 72/200
 - 1s - loss: 9.3776e-04 - val_loss: 0.0011
Epoch 73/200
 - 1s - loss: 9.3247e-04 - val_loss: 0.0011
Epoch 74/200
 - 1s - loss: 9.1912e-04 - val_loss: 0.0011
Epoch 75/200
 - 1s - loss: 9.1892e-04 - val_loss: 0.0011
Epoch 76/200
 - 1s - loss: 9.1309e-04 - val_loss: 0.0011

...

Epoch 187/200
 - 1s - loss: 5.5961e-04 - val_loss: 8.5856e-04
Epoch 188/200
 - 1s - loss: 5.5969e-04 - val_loss: 8.6770e-04
Epoch 189/200
 - 1s - loss: 5.5814e-04 - val_loss: 8.6158e-04
Epoch 190/200
 - 1s - loss: 5.5743e-04 - val_loss: 8.6480e-04
Epoch 191/200
 - 1s - loss: 5.5553e-04 - val_loss: 8.5850e-04
Epoch 192/200
 - 1s - loss: 5.5363e-04 - val_loss: 8.5433e-04
Epoch 193/200
 - 1s - loss: 5.5223e-04 - val_loss: 8.5982e-04
Epoch 194/200
 - 1s - loss: 5.5299e-04 - val_loss: 8.6184e-04
Epoch 195/200
 - 1s - loss: 5.5093e-04 - val_loss: 8.5972e-04
Epoch 196/200
 - 1s - loss: 5.4972e-04 - val_loss: 8.5229e-04
Epoch 197/200
 - 1s - loss: 5.4897e-04 - val_loss: 8.5745e-04
Epoch 198/200
 - 1s - loss: 5.4866e-04 - val_loss: 8.5599e-04
Epoch 199/200
 - 1s - loss: 5.4884e-04 - val_loss: 8.5532e-04
Epoch 200/200
 - 1s - loss: 5.4720e-04 - val_loss: 8.5943e-04
Train on 3912 samples, validate on 1304 samples
Epoch 1/200
 - 1s - loss: 0.0086 - val_loss: 0.0059
Epoch 2/200
 - 1s - loss: 0.0047

Epoch 138/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 139/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 140/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 141/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 142/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 143/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 144/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 145/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 146/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 147/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 148/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 149/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 150/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 151/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 152/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 153/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 154/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 155/200
 - 1s - loss: 0.0011 - val_loss: 0.0015
Epoch 156/200
 - 1s - loss: 

Epoch 91/200
 - 1s - loss: 0.0012 - val_loss: 0.0014
Epoch 92/200
 - 1s - loss: 0.0012 - val_loss: 0.0015
Epoch 93/200
 - 1s - loss: 0.0012 - val_loss: 0.0014
Epoch 94/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 95/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 96/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 97/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 98/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 99/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 100/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 101/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 102/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 103/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 104/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 105/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 106/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 107/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 108/200
 - 1s - loss: 0.0011 - val_loss: 0.0014
Epoch 109/200
 - 1s - loss: 0.0011 - 

Epoch 38/200
 - 1s - loss: 0.0014 - val_loss: 0.0015
Epoch 39/200
 - 1s - loss: 0.0014 - val_loss: 0.0014
Epoch 40/200
 - 1s - loss: 0.0014 - val_loss: 0.0014
Epoch 41/200
 - 1s - loss: 0.0014 - val_loss: 0.0014
Epoch 42/200
 - 1s - loss: 0.0014 - val_loss: 0.0014
Epoch 43/200
 - 1s - loss: 0.0014 - val_loss: 0.0014
Epoch 44/200
 - 1s - loss: 0.0014 - val_loss: 0.0014
Epoch 45/200
 - 1s - loss: 0.0014 - val_loss: 0.0014
Epoch 46/200
 - 1s - loss: 0.0014 - val_loss: 0.0014
Epoch 47/200
 - 1s - loss: 0.0013 - val_loss: 0.0014
Epoch 48/200
 - 1s - loss: 0.0013 - val_loss: 0.0014
Epoch 49/200
 - 1s - loss: 0.0013 - val_loss: 0.0014
Epoch 50/200
 - 1s - loss: 0.0013 - val_loss: 0.0014
Epoch 51/200
 - 1s - loss: 0.0013 - val_loss: 0.0013
Epoch 52/200
 - 1s - loss: 0.0013 - val_loss: 0.0013
Epoch 53/200
 - 1s - loss: 0.0013 - val_loss: 0.0013
Epoch 54/200
 - 1s - loss: 0.0013 - val_loss: 0.0013
Epoch 55/200
 - 1s - loss: 0.0013 - val_loss: 0.0013
Epoch 56/200
 - 1s - loss: 0.0013 - val_loss: 

Epoch 185/200
 - 1s - loss: 8.4375e-04 - val_loss: 9.7664e-04
Epoch 186/200
 - 1s - loss: 8.4232e-04 - val_loss: 9.7455e-04
Epoch 187/200
 - 1s - loss: 8.4070e-04 - val_loss: 9.7472e-04
Epoch 188/200
 - 1s - loss: 8.4433e-04 - val_loss: 9.8769e-04
Epoch 189/200
 - 1s - loss: 8.3896e-04 - val_loss: 9.7473e-04
Epoch 190/200
 - 1s - loss: 8.3687e-04 - val_loss: 9.7323e-04
Epoch 191/200
 - 1s - loss: 8.3371e-04 - val_loss: 9.7380e-04
Epoch 192/200
 - 1s - loss: 8.3370e-04 - val_loss: 9.6830e-04
Epoch 193/200
 - 1s - loss: 8.3060e-04 - val_loss: 9.6963e-04
Epoch 194/200
 - 1s - loss: 8.3036e-04 - val_loss: 9.6937e-04
Epoch 195/200
 - 1s - loss: 8.2781e-04 - val_loss: 9.7357e-04
Epoch 196/200
 - 1s - loss: 8.2552e-04 - val_loss: 9.6801e-04
Epoch 197/200
 - 1s - loss: 8.2330e-04 - val_loss: 9.6387e-04
Epoch 198/200
 - 1s - loss: 8.2357e-04 - val_loss: 9.6307e-04
Epoch 199/200
 - 1s - loss: 8.2231e-04 - val_loss: 9.6313e-04
Epoch 200/200
 - 1s - loss: 8.2205e-04 - val_loss: 9.6321e-04


In [76]:
print(result10)

0.154611570032655
